In [ ]:
import requests   # library to download files from internet

# the pdf link
url = "https://cgwb.gov.in/sites/default/files/inline-files/final_nhs-wq_pre_2023_compressed.pdf"

# the filename
filename ="ground_water_report2023.pdf"

# getting the file from the link
response = requests.get(url)

# checking if download was successful
if response.status_code == 200:
    # opening file in write-binary mode and saving
    with open(filename, "wb") as f:
        f.write(response.content)
    print("PDF downloaded and saved successfully!")
else:
    print("Failed to download. Error code:", response.status_code)


PDF downloaded and saved successfully!


In [ ]:
import pdfplumber
import camelot
import pandas as pd

pdf_path = "ground_water_report2023.pdf"
output_csv = "groundwaterreport.csv"

tables_collected = []
header = None  # to store the first header row

with pdfplumber.open(pdf_path) as pdf:
    num_pages = len(pdf.pages)
    print(f"Total pages: {num_pages}")

    for page_num in range(1, num_pages+1):
        print(f"Processing page {page_num}/{num_pages}…", end="")

        try:
            # Try Camelot extraction
            cam = camelot.read_pdf(pdf_path, pages=str(page_num), flavor="lattice")
            if cam.n == 0:
                cam = camelot.read_pdf(pdf_path, pages=str(page_num), flavor="stream")

            if cam.n == 0:
                print(" – no tables.")
                continue

            for idx, tbl in enumerate(cam):
                df = tbl.df.dropna(how="all", axis=0).dropna(how="all", axis=1)

                # Capture header only from the very first table
                if header is None:
                    header = df.iloc[0].tolist()
                    df = df[1:]
                else:
                    # Drop header rows that repeat
                    if df.iloc[0].tolist() == header:
                        df = df[1:]

                df.columns = header
                df["source_page"] = page_num
                tables_collected.append(df)

            print(f" – extracted {cam.n} tables.")

        except Exception as e:
            print(" – error:", e)

# Combine all tables
if tables_collected:
    final_df = pd.concat(tables_collected, ignore_index=True)
    final_df.to_csv("groundwaterreport.csv", index=False)
    print(f"\n✅ Clean extraction complete! Saved to {output_csv}")
    print("Rows:", df.shape[0], " | Columns:", df.shape[1])
else:

    print("\n⚠️ No tables extracted.")


Total pages: 200
Processing page 1/200… – extracted 1 tables.
Processing page 2/200… – extracted 1 tables.
Processing page 3/200… – extracted 1 tables.
Processing page 4/200… – extracted 1 tables.
Processing page 5/200… – extracted 1 tables.
Processing page 6/200… – extracted 1 tables.
Processing page 7/200… – extracted 1 tables.
Processing page 8/200… – extracted 1 tables.
Processing page 9/200… – extracted 1 tables.
Processing page 10/200… – extracted 1 tables.
Processing page 11/200… – extracted 1 tables.
Processing page 12/200… – extracted 1 tables.
Processing page 13/200… – extracted 1 tables.
Processing page 14/200… – extracted 1 tables.
Processing page 15/200… – extracted 1 tables.
Processing page 16/200… – extracted 1 tables.
Processing page 17/200… – extracted 1 tables.
Processing page 18/200… – extracted 1 tables.
Processing page 19/200… – extracted 1 tables.
Processing page 20/200… – extracted 1 tables.
Processing page 21/200… – extracted 1 tables.
Processing page 22/200… – 